In [1]:
import pandas as pd
import requests
import cv2
import random
from tqdm import tqdm

# Load the dataset
train_file = "dataset/train.csv"
test_file = "dataset/test.csv"
constants_file = "src/constants.py"

In [2]:

# Load train and test data
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

# Question 1: Check if the image links are working
def check_image_links(data, n_samples=100):
    working_links = []
    for i, url in enumerate(tqdm(data['image_link'].sample(n_samples))):
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                working_links.append(True)
            else:
                working_links.append(False)
        except:
            working_links.append(False)
    return working_links

# Question 2: Extract unique entity names and check consistency of entity values
def check_entity_consistency(train_data):
    entity_types = train_data['entity_name'].unique()
    print(f"Unique entity names: {entity_types}")
    entity_values = train_data.groupby('entity_name')['entity_value'].apply(lambda x: x.unique())
    return entity_values

# Question 3: List of allowed units from constants.py
def extract_allowed_units(file_path):
    with open(file_path, 'r') as f:
        data = f.read()
    allowed_units = []
    # Assuming the units are in a dictionary format
    lines = data.split('\n')
    for line in lines:
        if "=" in line:
            key, value = line.split("=")
            allowed_units.append(value.strip())
    return allowed_units

# Question 4: Check if data has any inconsistencies/missing values
def check_data_issues(data):
    missing_values = data.isnull().sum()
    return missing_values

# Updated sampling function to handle small datasets
def sample_test_data(train_data, test_data, n_samples=100):
    train_size = len(train_data)
    test_size = len(test_data)

    # Ensure we don't sample more than available rows
    train_sample_size = min(train_size, n_samples)
    test_sample_size = min(test_size, n_samples)

    # Sampling train and test data
    train_sample = train_data.sample(train_sample_size)
    test_sample = test_data.sample(test_sample_size)

    return train_sample, test_sample

# Call this new function
train_sample, test_sample = sample_test_data(train_data, test_data)

# Function to run all checks
def run_all_checks(train_data, test_data, constants_file):
    print("Checking if image links are working...")
    image_link_status = check_image_links(train_data)
    print(f"Image link working percentage: {sum(image_link_status) / len(image_link_status) * 100}%")

    print("Extracting entity consistency info...")
    entity_values = check_entity_consistency(train_data)
    print("Unique entity values by entity name:")
    print(entity_values)

    print("Checking for missing values...")
    train_missing = check_data_issues(train_data)
    test_missing = check_data_issues(test_data)
    print(f"Missing values in training data:\n{train_missing}")
    print(f"Missing values in test data:\n{test_missing}")

    print("Extracting allowed units from constants.py...")
    allowed_units = extract_allowed_units(constants_file)
    print(f"Allowed units: {allowed_units}")

    print("Sampling 100 rows for testing...")
    train_sample, test_sample = sample_test_data(train_data, test_data)
    print(f"Train sample:\n{train_sample.head()}")
    print(f"Test sample:\n{test_sample.head()}")

    return train_sample, test_sample

# Run the checks
train_sample, test_sample = run_all_checks(train_data, test_data, constants_file)


Checking if image links are working...


100%|██████████| 100/100 [03:18<00:00,  1.99s/it]


Image link working percentage: 100.0%
Extracting entity consistency info...
Unique entity names: ['item_weight' 'item_volume' 'voltage' 'wattage'
 'maximum_weight_recommendation' 'height' 'depth' 'width']
Unique entity values by entity name:
entity_name
depth                            [21.0 centimetre, 20.5 centimetre, 63.0 millim...
height                           [95.0 centimetre, 46.5 millimetre, 71.0 centim...
item_volume                      [1.0 cup, 4.0 gallon, 10.0 ounce, 30.0 millili...
item_weight                      [500.0 gram, 0.709 gram, 1400 milligram, 30.0 ...
maximum_weight_recommendation    [15 kilogram, 350 pound, 120 kilogram, 7 kilog...
voltage                          [48.0 volt, 36.0 volt, [100.0, 240.0] volt, 12...
wattage                          [800.0 watt, 150.0 watt, 30.0 watt, 250.0 watt...
width                            [22.0 millimetre, 20.0 centimetre, 9.0 centime...
Name: entity_value, dtype: object
Checking for missing values...
Missing values in

In [ ]:
!pip install pytesseract


In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import cv2
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import pytesseract
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import random

# Set random seed for reproducibility
random.seed(42)


In [ ]:
!apt-get install tesseract-ocr -y


In [4]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Tesseract-OCR\tesseract.exe'


In [ ]:
!pip install tqdm


In [5]:
import numpy as np
import pandas as pd
import cv2
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import pytesseract
from tqdm import tqdm

# Load sampletrain.csv data
train_df = pd.read_csv("dataset/train.csv")
test_df = pd.read_csv("dataset/test.csv")

# Function to download and preprocess image
def load_image_from_url(image_url):
    try:
        response = requests.get(image_url)
        # Check if the response is valid and the content type is an image
        if response.status_code == 200 and 'image' in response.headers['Content-Type']:
            img = Image.open(BytesIO(response.content))
            img = img.resize((128, 128))  # Resizing images to standard size for CNN
            img = np.array(img)
            if img.shape[-1] == 4:  # Handle transparency in PNG
                img = img[..., :3]
            return img / 255.0  # Normalize image
        else:
            print(f"Invalid image URL or non-image content at: {image_url}")
            return np.zeros((128, 128, 3))  # Return a black image as a placeholder
    except UnidentifiedImageError:
        print(f"UnidentifiedImageError for URL: {image_url}")
        return np.zeros((128, 128, 3))  # Return a black image as a placeholder
    except Exception as e:
        print(f"Error downloading image from {image_url}: {e}")
        return np.zeros((128, 128, 3))  # Return a black image as a placeholder

# Function to extract text using OCR
def extract_text_from_image(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200 and 'image' in response.headers['Content-Type']:
            img = Image.open(BytesIO(response.content))
            text = pytesseract.image_to_string(img)
            return text
        else:
            print(f"Invalid image URL or non-image content at: {image_url}")
            return ""  # Return empty string for non-image content
    except UnidentifiedImageError:
        print(f"UnidentifiedImageError for URL: {image_url}")
        return ""  # Return empty string for invalid images
    except Exception as e:
        print(f"Error extracting text from {image_url}: {e}")
        return ""  # Return empty string in case of error

# Apply to the dataset with progress bar
tqdm.pandas()  # Enable progress_apply for pandas

train_df['image_data'] = train_df['image_link'].progress_apply(load_image_from_url)
train_df['ocr_text'] = train_df['image_link'].progress_apply(extract_text_from_image)

test_df['image_data'] = test_df['image_link'].progress_apply(load_image_from_url)
test_df['ocr_text'] = test_df['image_link'].progress_apply(extract_text_from_image)


  0%|          | 522/263859 [06:43<58:45:28,  1.24it/s]

Invalid image URL or non-image content at: https://m.media-amazon.com/images/I/1yw53vfQtS.jpg


  0%|          | 1098/263859 [13:56<79:31:38,  1.09s/it]

Error downloading image from https://m.media-amazon.com/images/I/41NH8WgeBOL.jpg: image file is truncated (2 bytes not processed)


  1%|          | 1764/263859 [22:03<52:27:39,  1.39it/s]

In [ ]:
# Data Augmentation to boost the number of images for certain entities
data_gen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Sample augmented images
image_sample = train_df['image_data'].iloc[0].reshape((1, 128, 128, 3))  # Sample image for augmentation
augmented_images = data_gen.flow(image_sample, batch_size=1)

# Plot augmented images
for i in range(5):
    augmented_image = next(augmented_images)[0]
    plt.imshow(augmented_image)
    plt.show()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))  # Output a numeric value (like item_weight)
    return model

# Instantiate and compile the model
cnn_model = create_cnn_model()
cnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Convert images for CNN training
X_images = np.stack(train_df['image_data'].values)

# Ensure y_values is numeric
y_values = pd.to_numeric(train_df['entity_value'], errors='coerce')

# Check and handle NaNs if necessary
if y_values.isna().any():
    print("Warning: Some values in 'entity_value' could not be converted to numeric.")
    # Optionally handle NaNs here (e.g., impute or remove them)
    y_values = y_values.fillna(0)  # Example: fill NaNs with 0

# Convert to numpy array
y_values = y_values.values

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_images, y_values, test_size=0.2, random_state=42)

# Ensure the shapes are correct
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

# Ensure y_train and y_val are float32
y_train = y_train.astype('float32')
y_val = y_val.astype('float32')

# Train the CNN model
cnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)


In [ ]:
# Use the OCR output for text-based model
def text_based_model(ocr_text):
    # Simple model that analyzes OCR text (could be more sophisticated)
    if 'gram' in ocr_text.lower():
        return 'weight'
    elif 'volt' in ocr_text.lower():
        return 'voltage'
    elif 'watt' in ocr_text.lower():
        return 'wattage'
    # Add more entity handling rules as needed
    return None

# Apply the OCR-based model on the dataset
train_df['predicted_entity'] = train_df['ocr_text'].apply(text_based_model)

# Convert text-based predictions to numerical values for ensemble
def entity_to_numeric(entity):
    # Mapping text-based entities to simple numeric placeholders
    entity_map = {'weight': 1.0, 'voltage': 2.0, 'wattage': 3.0}
    return entity_map.get(entity, 0.0)  # Default to 0 if entity not found

train_df['predicted_entity_numeric'] = train_df['predicted_entity'].apply(entity_to_numeric)

# Ensemble predictions: average CNN model output and OCR predictions
cnn_predictions = cnn_model.predict(X_images).flatten()

# Ensure both columns are numeric before averaging
train_df['ensemble_pred'] = (train_df['predicted_entity_numeric'] + cnn_predictions) / 2


In [ ]:
import numpy as np
import requests
from PIL import Image
from io import BytesIO

def load_image_from_url(image_url, target_size=(128, 128)):
    try:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        img = img.convert('RGB')  # Ensure image is RGB
        img = img.resize(target_size)  # Resize images to standard size
        img = np.array(img)
        return img / 255.0  # Normalize image
    except Exception as e:
        print(f"Error loading image from {image_url}: {e}")
        # Return a blank image in case of an error
        return np.zeros((*target_size, 3))

def preprocess_images(image_urls, target_size=(128, 128)):
    images = []
    for url in image_urls:
        img = load_image_from_url(url, target_size)
        if img.shape != (*target_size, 3):
            print(f"Unexpected image shape: {img.shape}")
            img = np.zeros((*target_size, 3))  # Replace with a blank image if shape is incorrect
        images.append(img)
    images_array = np.array(images)
    return images_array

# Example usage for test data
test_image_urls = test_df['image_link'].values  # Assuming image URLs are in 'image_link'
test_images = preprocess_images(test_image_urls)
print(f"Shape of test_images: {test_images.shape}")



In [ ]:
import cv2
import numpy as np

def preprocess_and_stack_images(image_data_list, target_size=(128, 128)):
    """
    Resize and preprocess images to ensure they all have the same shape.

    Args:
        image_data_list (list of numpy arrays): List of image data arrays.
        target_size (tuple): Target size for the images (height, width).

    Returns:
        numpy array: Stacked array of preprocessed images.
    """
    processed_images = []

    for img in image_data_list:
        # Check if img is grayscale (2D) or RGB (3D)
        if img.ndim == 2:  # Grayscale image
            img = np.stack([img] * 3, axis=-1)  # Convert grayscale to RGB
        elif img.ndim == 3 and img.shape[-1] == 4:  # RGBA image
            img = img[..., :3]  # Remove alpha channel
        elif img.ndim != 3 or img.shape[-1] not in [1, 3]:  # Unexpected number of channels
            raise ValueError(f"Unexpected image format. Expected grayscale or RGB with 1 or 3 channels, got {img.ndim}D with shape {img.shape}.")

        # Resize image
        img_resized = cv2.resize(img, target_size[::-1])  # Resize to target size (width, height)

        # Check image has 3 channels
        if img_resized.shape[-1] != 3:
            raise ValueError(f"Unexpected number of channels in resized image. Expected 3, got {img_resized.shape[-1]}.")

        processed_images.append(img_resized)

    return np.array(processed_images)

# Ensure test images have the same shape
test_image_data_list = test_df['image_data'].values
test_images = preprocess_and_stack_images(test_image_data_list)

# Predictions on test data
cnn_predictions = cnn_model.predict(test_images).flatten()

# Ensure y_true and y_pred are numeric and handle NaNs if necessary
y_true = pd.to_numeric(train_df['entity_value'], errors='coerce').fillna(0).values
y_pred = pd.to_numeric(train_df['ensemble_pred'], errors='coerce').fillna(0).values

# Ensure the predictions and true values are compatible for F1 score calculation
# If you are using regression, the F1 score might not be suitable; consider other metrics
if len(y_true) == len(y_pred):
    # For demonstration, using rounded values for binary classification example
    f1 = f1_score(y_true.round(), y_pred.round(), average='macro')
    print(f'Final F1 Score: {f1}')
else:
    print('Mismatch in length of true values and predictions.')
